In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
# filepath = 'xlsdir/train_all_int.xls'

# df = pd.read_excel(filepath, skiprows=70000)

In [3]:
# print(len(df))
# dds = df.head()
# print(dds)
# print(len(dds))
# df.dtypes

In [4]:
# 数据进行切割，第一列切割为Y，其余列切割为X
# print("第一列", dds.iloc[:, :1])

In [5]:
f = open("xlsdir/train_all.csv","r")
lines = f.readlines()

In [6]:
X = []
Y = []
max_columns_count = 0
max_line_length = 0
total_columns_count = 0
x_columns_limit = 1000
for i in range(len(lines)):
    # if i > 20:
    #     break
    line = lines[i]
    line_length = len(line)
    if line_length > max_line_length:
        max_line_length = line_length
    line_arr = line.split(',')
    x_arr = line_arr[1:]
    y = line_arr[:1]
    x_columns_count = len(x_arr)
    total_columns_count += x_columns_count
    while x_columns_count > x_columns_limit:
        X.append(x_arr[:x_columns_limit])
        Y.append(y)
        x_arr = x_arr[x_columns_limit:]
        x_columns_count = len(x_arr)
        if x_columns_count > max_columns_count:
            max_columns_count = x_columns_count
    # total_columns_count += x_columns_count

    X.append(x_arr)
    x_columns_count = len(x_arr)
    if x_columns_count > max_columns_count:
        max_columns_count = x_columns_count
    Y.append(y)
    # print(lines[i])
    # for s in line_arr:
    #     n = int(s)
        # print(int(s))
f.close()

In [7]:
X_2 = []
for arr in X:
    arr_2 = []
    for s in arr:
        i = int(s)
        arr_2.append(i)
    X_2.append(arr_2)
X = X_2

In [8]:
max_columns_count = 0
for arr in X:
    if len(arr) > max_columns_count:
        max_columns_count = len(arr)

In [9]:
print(len(X))
print(len(Y))
print(max_columns_count)
print(max_line_length)
print(total_columns_count/len(X))

103123
103123
1000
43822
266.73552941632806


In [10]:
def arr_append_0(append_arr, append_limit):
    arr_length = len(append_arr)
    if arr_length < append_limit:
        append_range = append_limit - len(append_arr)
        for _ in range(append_range):
            append_arr.append(0)
    return append_arr

In [11]:
# 数组补0

for arr in X:
    arr = arr_append_0(arr, x_columns_limit)

for arr in X[:10]:
    print(len(arr))

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000


In [12]:
# 103123 总长度
# X dim 1000
# Y dim 1

# 其中划分80000，做训练集
# 划分10000，做测试集
# 划分10000，做验证集
# 循环训练
total_row_count = len(X)
train_row_count = 80000
test_row_count = 10000
validate_row_count = total_row_count - train_row_count - test_row_count

In [13]:
# Y转换为枚举类型
# info:1
# warn:2
# debug:3
# error:4
# other:5
Y_2 = []
for s in Y:
    i = 5
    if 'info' == s[0].strip().lower():
       i = 1
    elif 'warn' == s[0].strip().lower():
       i = 2
    elif 'debug' == s[0].strip().lower():
       i = 3
    elif 'error' == s[0].strip().lower():
       i = 4
    Y_2.append(i)
Y = Y_2

In [46]:
X_train = X[:train_row_count + test_row_count]
X_test = X[train_row_count:train_row_count + test_row_count]
X_validate = X[train_row_count + test_row_count:]
X_train_data = tf.convert_to_tensor(X_train)
print(X_train_data.shape)

(90000, 1000)


In [15]:
X_test_data = tf.convert_to_tensor(X_test)
X_validate_data = tf.convert_to_tensor(X_validate)

In [47]:
Y_train = Y[:train_row_count + test_row_count]
Y_train_data = tf.convert_to_tensor(Y_train)

In [48]:
one_hot_depth = 5
# 不做转型会报 Could not find valid device for node 的异常
Y_train_data = tf.cast(Y_train_data, tf.int32)
Y_train_data = tf.one_hot(Y_train_data, one_hot_depth)

In [49]:
Y_test = Y[train_row_count:train_row_count + test_row_count]
Y_test_data = tf.convert_to_tensor(Y_test)
Y_validate = Y[train_row_count + test_row_count:]
Y_validate_data = tf.convert_to_tensor(Y_validate)



In [50]:
# 不做转型会报 Could not find valid device for node 的异常
Y_test_data = tf.cast(Y_test_data, tf.int32)
Y_test_data = tf.one_hot(Y_test_data, one_hot_depth)

# 不做转型会报 Could not find valid device for node 的异常
Y_validate_data = tf.cast(Y_validate_data, tf.int32)
Y_validate_data = tf.one_hot(Y_validate_data, one_hot_depth)

In [20]:
from tensorflow import keras
from tensorflow.keras import layers



In [43]:
# 设置checkpoint
check_point_path = 'checkpoint/train_1'
import os
if not os.path.exists(check_point_path) or os.path.isfile(check_point_path):
    os.mkdir(check_point_path)
    


In [52]:

train_1_callback = keras.callbacks.ModelCheckpoint(
    filepath=check_point_path,
    # save_best_only=True,
    save_weights_only=True,

)

In [55]:
def gru_model():
    model = keras.Sequential([
        layers.Embedding(input_dim=1000,
                         output_dim=32,
                         input_length=1000,
                         ),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.Dense(
            512,
            activation='ReLU'
        ),
        layers.Dropout(0.5),
        layers.BatchNormalization(),
        layers.GRU(32, return_sequences=True),
        layers.BatchNormalization(),
        layers.GRU(one_hot_depth, activation='sigmoid', return_sequences=False)
    ])
    model.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.BinaryCrossentropy(),
                 metrics=['accuracy'])
    return model


In [56]:
model = gru_model()
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 1000, 32)          1307808   
_________________________________________________________________
gru_8 (GRU)                  (None, 1000, 32)          6336      
_________________________________________________________________
gru_9 (GRU)                  (None, 5)                 585       
Total params: 1,314,729
Trainable params: 1,314,729
Non-trainable params: 0
_________________________________________________________________


In [ ]:
stop_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')


In [ ]:
%%time
history = model.fit(
    X_train_data,
    Y_train_data,
    batch_size=64,
    epochs=10,
    validation_split=0.1,
    callbacks=[train_1_callback, stop_callback]
)


Epoch 1/10
1266/1266 [==============================] - 1212s 958ms/step - loss: 0.1648 - accuracy: 0.8400 - val_loss: 0.0354 - val_accuracy: 0.9709
Epoch 2/10
1266/1266 [==============================] - 1218s 962ms/step - loss: 0.0282 - accuracy: 0.9818 - val_loss: 0.0151 - val_accuracy: 0.9947
Epoch 3/10
1266/1266 [==============================] - 1220s 964ms/step - loss: 0.0158 - accuracy: 0.9900 - val_loss: 0.0471 - val_accuracy: 0.9768
Epoch 4/10
1266/1266 [==============================] - 1218s 962ms/step - loss: 0.0136 - accuracy: 0.9912 - val_loss: 0.0075 - val_accuracy: 0.9990
Epoch 5/10
1266/1266 [==============================] - 1219s 963ms/step - loss: 0.0105 - accuracy: 0.9929 - val_loss: 0.0094 - val_accuracy: 0.9983
Epoch 6/10
1266/1266 [==============================] - 1218s 962ms/step - loss: 0.0101 - accuracy: 0.9930 - val_loss: 0.0440 - val_accuracy: 0.9596
Epoch 7/10
1266/1266 [==============================] - 1220s 964ms/step - loss: 0.0076 - accuracy: 0.9950

In [30]:
Y_predict = model.predict(X_validate_data)

In [31]:
print(Y_predict.shape)

(13123, 5)


In [36]:
list_1 = Y_predict.tolist()
print(list_1[:10])

[[6.036349441274069e-05, 0.9958725571632385, 0.0026994943618774414, 5.295994196785614e-05, 0.004691628273576498], [2.9773944334010594e-05, 0.003400536486878991, 0.010829447768628597, 0.9982026219367981, 0.016583293676376343], [5.6502198276575655e-05, 0.9945985674858093, 0.0032556354999542236, 8.043862180784345e-05, 0.005636726040393114], [6.049039075151086e-05, 0.995907187461853, 0.002684086561203003, 5.308178879204206e-05, 0.004665063694119453], [2.980397221108433e-05, 0.003366216318681836, 0.010833054780960083, 0.9984875321388245, 0.016582727432250977], [5.761990541941486e-05, 0.9950159788131714, 0.003075510263442993, 7.571928290417418e-05, 0.005333354230970144], [2.980385943374131e-05, 0.003366404678672552, 0.010833293199539185, 0.9984781742095947, 0.016583174467086792], [5.764815796283074e-05, 0.9950259327888489, 0.0030711591243743896, 7.458451727870852e-05, 0.005326045677065849], [6.0811988078057766e-05, 0.9959931373596191, 0.0026457607746124268, 5.22411210113205e-05, 0.0045988410

In [32]:
ar_1 = [1, 0, 0]
ar_2 = [1, 0, 0]
print(ar_1 == ar_2)

True


In [35]:
Y_p = Y_predict.tolist()
Y_o = Y_validate_data.numpy().tolist()
all_count = len(Y_p)
correct_count = 0
for i in range(all_count):
    Y_p_arr = Y_p[i]
    Y_o_arr = Y_o[i]
    if Y_p_arr == Y_o_arr:
        correct_count += 1
print(correct_count/all_count)

0.0


In [38]:
loss,accuracy = model.evaluate(X_validate_data,Y_validate_data)
print('\ntest loss',loss)
print('accuracy',accuracy)

411/411 [==============================] - 52s 126ms/step - loss: 0.0420 - accuracy: 0.9803

test loss 0.0419534407556057
accuracy 0.9803398847579956


In [39]:
def cudnn_gru_model():
    model = keras.Sequential([
        layers.Embedding(input_dim=40869, output_dim=32, input_length=1000),
        layers.CuDNNGRU(32, return_sequences=True),
        layers.CuDNNGRU(1, activation='sigmoid', return_sequences=False)
    ])
    model.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.BinaryCrossentropy(),
                 metrics=['accuracy'])
    return model
model = cudnn_gru_model()
model.summary()

AttributeError: module 'tensorflow.keras.layers' has no attribute 'CuDNNGRU'